# IG360 QA - Create Full Profile
*Purpose:* Create a full profile for a given user:
<ul>
    <li>User name, full name</li>
    <li>Stats on followers, following</li>
    <li>Recent posts</li>
</ul>
<br>
<br>
Import libraries, set script configurations

In [ ]:
# import IG360 data
from IG360.ig360_scrape import *

# import other key libraries
import pandas as pd
import datetime
import json
import re
from pathlib import Path

# set IG login and password
ig_login = "xx"
ig_pass = "xx"

# set output folder
txt_dir_output = '_output/'
dir_output = Path(txt_dir_output)

# locate configuration file
config_file_path = 'config.ini'
config = init_config(config_file_path)

# set target account and capture parameters
tgt_user = "ghost.acolyte"
num_posts = 100

Initialize Scraper and login

In [ ]:
scraper = IG360Scrape(config, ig_login,ig_pass)
scraper.login()

Scrape profile

In [ ]:
scraper.scrape_profile(tgt_user)
print(scraper.record_profile)

Get followers / following

In [ ]:
scraper.scrape_followers(tgt_user)
scraper.scrape_following(tgt_user)
print("Followers: {}".format(scraper.record_follower_list[:10]))
print("Following: {}".format(scraper.record_following_list[:10]))

Get most recent posts

In [ ]:
scraper.scrape_post_list(tgt_user, num_posts)
print("Posts Found: {}".format(scraper.record_post_list))
print("Num Posts: {}".format(len(scraper.record_post_list)))

Combine data into a single dictionary

In [ ]:
# combine follow lists into single dictionary
dict_lists = {
    "followers" : scraper.record_follower_list,
    "user_follows" : scraper.record_following_list,
    "recent_posts" : scraper.record_post_list
}

# create header dictionary
dt_run = datetime.datetime.now()
dict_header = {
    "run_datetime" : dt_run.strftime('%Y-%m-%dT%H:%M:%SZ'),
    "ig_record_type" : "fp",
    "code_version" : scraper.code_version
}

# combine all dictionaries
dict_final = {**dict_header, **scraper.record_profile, **dict_lists}

# output as JSON
fn_out = re.sub('[\W]','',dict_header["ig_record_type"]) + '_' + tgt_user + '_' \
    + str(dt_run.year) + str(dt_run.month).zfill(2) + str(dt_run.day).zfill(2) + str(dt_run.hour).zfill(2) + str(dt_run.minute).zfill(2) + '.json'
fn_out_full = dir_output / fn_out
with open(fn_out_full, 'w') as json_file:
    json.dump(dict_final, json_file)


In [ ]:
fn_out_full